<a href="https://colab.research.google.com/github/Paul-Richmond/hepthLlama/blob/main/vLLM_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip installs and imports

In [4]:
!pip install -q vllm # takes a few minutes to install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/6

In [5]:
!pip install bitsandbytes>=0.44.0

In [6]:
import huggingface_hub
import os
import re
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
from datasets import load_dataset

# Login to Huggingface hub

In [7]:
huggingface_hub.login()

# Definitions

In [8]:
dataset_name = 'LLMsForHepth/hep-th_primary'
base_model_name = 'meta-llama/Meta-Llama-3.1-8B'
lora_adapter_name = 'LLMsForHepth/s1-L-3.1-8B-base'

generation_cfg = {'max_tokens': 1024,
                  'min_tokens': 48}

model_cfg = {'dtype': 'half',
             'quantization': "bitsandbytes",
             'load_format': "bitsandbytes",
             'max_model_len': 1024,
             'gpu_memory_utilization': 0.6}

In [9]:
def split_abstracts(example):
    """
    Splits an abstract into a prompt and ground truth.

    The prompt is created from the first half (or slightly more) of the sentences in the abstract,
    and the ground truth is the remaining sentences.
    If there is only a single sentence then we split instead on spaces to get, roughly, words
    and take the first half (or slightly more) of these.

    Args:
        example (dict): A dictionary containing the 'abstract' text to be split.

    Returns:
        dict: A dictionary with 'prompt' and 'y_true' keys containing the split abstract parts.
    """
    text = example['abstract']
    # Split the abstract into sentences (i.e. text sequences which end with any of .!? and a space)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Calculate the split point
    total_sentences = len(sentences)
    if total_sentences > 1:  # more than 1 sentence so can split
        split_point = (total_sentences + 1) // 2  # Ensures the prompt has >= number of sentences than y_true
        # Join the sentences back into two parts
        prompt = ' '.join(sentences[:split_point])
        y_true = ' '.join(sentences[split_point:])
    else:  # only a single sentence so split on words (latex commands between $$ might get split)
        words = text.split()
        total_words = len(words)
        split_point = (total_words + 1) // 2  # Ensures the prompt has >= number of sentences than y_true
        # Join the sentences back into two parts
        prompt = ' '.join(words[:split_point])
        y_true = ' '.join(words[split_point:])
    return {'prompt': prompt, 'y_true': y_true}

# Load dataset and pre-process

In [10]:
ds = load_dataset(dataset_name, split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/80.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/73768 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15808 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15808 [00:00<?, ? examples/s]

In [11]:
ds = ds.remove_columns(list(set(ds.column_names) - {'id', 'abstract'}))
ds = ds.map(split_abstracts,
            batched=False,
            desc='Splitting abstracts')

Splitting abstracts:   0%|          | 0/15808 [00:00<?, ? examples/s]

In [12]:
prompts = ds['prompt'][:10] # slice for smaller list

# Load LLM and generate

In [13]:
if lora_adapter_name is not None:
    enable_lora = True
    lora_request = LoRARequest("sql_adapter", 1, lora_path=lora_adapter_name)
else:
    enable_lora = False
    lora_request = None

if generation_cfg is not None:
    sampling_params = SamplingParams(**generation_cfg)
else:
    sampling_params = SamplingParams()

# Load the model
llm = LLM(model=base_model_name, enable_lora=enable_lora, **model_cfg)
# Generate completions
outputs = llm.generate(prompts, sampling_params, lora_request=lora_request)

<ipython-input-13-832d2cd02c0e>:3: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request = LoRARequest("sql_adapter", 1, lora_path=lora_adapter_name)


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

WARNING 10-18 12:33:44 config.py:1668] Casting torch.bfloat16 to torch.float16.
WARNING 10-18 12:33:55 config.py:321] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 10-18 12:33:55 config.py:1562] bitsandbytes quantization is not tested with LoRA yet.
INFO 10-18 12:33:55 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

scheduler.pt:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

last-checkpoint/trainer_state.json:   0%|          | 0.00/3.22M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.82k [00:00<?, ?B/s]

Processed prompts: 100%|██████████| 10/10 [03:29<00:00, 20.99s/it, est. speed input: 5.10 toks/s, output: 6.11 toks/s]


In [14]:
completions = []
for output in outputs:
    completions.append(output.outputs[0].text)

In [15]:
prompts[0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation.'

In [16]:
prompts[0] + completions[0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation.yboard-1 is exactly the well-known section constraint, while the other two cmds are a master equation and the vector constraint potential together with an expression for the field strengths. In the physical r chamber all sections of the particular E7 reduction are closed by the action of and satisfy the E7 Lie derivatives.'

In [17]:
ds['abstract'][0]

'We demonstrate that exceptional field theory is a truncation of the non-linear realisation of the semi-direct product of E11 and its first fundamental as proposed in 2003. Evaluating the simple equations of the E11 approach, and using the commutators of the E11 algebra, we find the equations of exceptional field theory after making a radical truncation. This procedure does not respect any of the higher level E11 symmetries and so these are lost. We suggest that the need for the section condition in exceptional field theory could be a consequence of the truncation.'

# Change temperature

In [24]:
temps = [0.1 * i for i in range(11)]

prompt = prompts[0]
for temp in temps:
    generation_cfg['temperature'] = temp
    sampling_params = SamplingParams(**generation_cfg)
    outputs = llm.generate(prompt, sampling_params, lora_request=lora_request)
    print(f'Temperature: {temp} \n')
    print(outputs[0].outputs[0].text)
    print('-' * 20)

Processed prompts: 100%|██████████| 1/1 [01:06<00:00, 66.92s/it, est. speed input: 1.12 toks/s, output: 14.20 toks/s]


Temperature: 0.0 

://www.maths.ex.ac.uk/research/grp/e11/ (http://www.maths.ex.ac.uk/research/grp/e11/) 2003.01.31 (v1) 2003.02.03 (v2) 2003.02.05 (v3) 2003.02.07 (v4) 2003.02.10 (v5) 2003.02.12 (v6) 2003.02.14 (v7) 2003.02.17 (v8) 2003.02.19 (v9) 2003.02.21 (v10) 2003.02.24 (v11) 2003.02.26 (v12) 2003.02.28 (v13) 2003.03.03 (v14) 2003.03.05 (v15) 2003.03.07 (v16) 2003.03.10 (v17) 2003.03.12 (v18) 2003.03.14 (v19) 2003.03.17 (v20) 2003.03.19 (v21) 2003.03.21 (v22) 2003.03.24 (v23) 2003.03.26 (v24) 2003.03.28 (v25) 2003.03.31 (v26) 2003.04.02 (v27) 2003.04.04 (v28) 2003.04.07 (v29) 2003.04.09 (v30) 2003.04.11 (v31) 2003.04.14 (v32) 2003.04.16 (v33) 2003.04.18 (v34) 2003.04.21 (v35) 2003.04.23 (v36) 2003.04.25 (v37) 2003.04.28 (v38) 2003.04.30 (v39) 2003.05.02 (v40) 2003.05.04 (v41) 2003.05.06 (v42) 2003.05.08 (v43) 2003.05.11 (v44) 2003.05.13 (v45) 2003.05.15 (v46) 2003.05.17 (v47) 2003.05.19 (v48) 2003.05.21 (v49) 2003.05.23 (v50) 2003.05.26 (v51) 2003.05.28 (v52) 2003.05.30 (v53) 200

Processed prompts: 100%|██████████| 1/1 [01:09<00:00, 69.56s/it, est. speed input: 1.08 toks/s, output: 13.66 toks/s]


Temperature: 0.1 

://www.maths.ex.ac.uk/research/grp/e11/ (http://www.maths.ex.ac.uk/research/grp/e11/) 2003.09.09: first version of this paper. 2003.10.09: minor changes. 2003.10.15: minor changes. 2003.10.16: minor changes. 2003.10.17: minor changes. 2003.10.18: minor changes. 2003.10.20: minor changes. 2003.10.21: minor changes. 2003.10.22: minor changes. 2003.10.23: minor changes. 2003.10.24: minor changes. 2003.10.25: minor changes. 2003.10.26: minor changes. 2003.10.27: minor changes. 2003.10.28: minor changes. 2003.10.29: minor changes. 2003.10.30: minor changes. 2003.10.31: minor changes. 2003.11.01: minor changes. 2003.11.02: minor changes. 2003.11.03: minor changes. 2003.11.04: minor changes. 2003.11.05: minor changes. 2003.11.06: minor changes. 2003.11.07: minor changes. 2003.11.08: minor changes. 2003.11.09: minor changes. 2003.11.10: minor changes. 2003.11.11: minor changes. 2003.11.12: minor changes. 2003.11.13: minor changes. 2003.11.14: minor changes. 2003.11.15: minor

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.66s/it, est. speed input: 16.11 toks/s, output: 11.17 toks/s]


...

Temperature: 0.8 

:// COPSEVZMST 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.51s/it, est. speed input: 7.89 toks/s, output: 12.20 toks/s]


Temperature: 0.9 

_REF_0125]{\R}{\R}{\R}{\R}{\R} provides some evidence that this is the most general consistent truncation. The results are a consequence of a representation of the E11 algebra on string Lie 3-algebras. We also show that the E11 representation is significantly constrained by closure of the E11 commutators, suggesting that only simple extensions of the truncation to include more higher rank field are possible. This is the complete reformulation of exceptional field theory using the formalism of non-linear realisation.
--------------------


Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.72s/it, est. speed input: 13.12 toks/s, output: 11.02 toks/s]

Temperature: 1.0 

.TextField:@European Physical Journal C 69 @, 183 (2010).Hear(E1x)Context: hep-th/hep-th/0405060+ [Mathematics] linear systemshep-th/0601054+ Fieldsell angular momentum vectorshep-th/0612170+
--------------------
